In [1]:
from ipywidgets import widgets
from IPython.display import display

In [2]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.widgets
init_notebook_mode(connected=True)
plot_config={'showLink':False}

<IPython.core.display.Javascript object>

In [6]:
import json
with open('heatmapdata.json') as f:
    offlinedata=json.load(f)
hits = offlinedata['hits']['hits']

In [7]:
#Slider

atc_slider=widgets.SelectionSlider(
    options=[1,3,4,5,7],
    value=1,
    description='Anatomic Level:',
    style={'description_width':'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [10]:
import pandas as pd
from pandas.io.json import json_normalize

In [141]:
def flatify(json):

    
    def engine(data, header=''):
        if type(data) is list:
            c=0
            for obj in data:
                engine(obj, '{}[{}]'.format(header,str(c)))
                c+=1
                
        elif type(data) is dict:
            for k,v in data.items():
                engine(v, header+k+'.')
        
        else:
            row[header]=data
    row={}            
    engine(json)
    return row

In [149]:
#data = [hits[0]['_source'],hits[1]['_source']]
data = hits[8607]['_source']
r = flatify(data)
for k,v in r.items():
    print(k, v)

pm_date. 2018-09-27T00:00:00
status_current_f. Commercialisé
pharmaceutical_std.[0] MFR
product_monograph_en_url. https://pdf.hres.ca/dpd_pm/00048521.PDF
ai_group_no. 0154610002
dosage_form_f.[0] Solution
active_ingredients.[0] OCRIPLASMIN
drug_code. 93860
extract. active
route_f.[0] Intravitreuse
schedule.[0] Schedule D
number_of_ais. 1
risk_man_plan. N
status_approved_date.[0] 2016-02-16T00:00:00
schedule_f.[0] Annexe D
brand_name. JETREA
dosage_form.[0] Solution
product_monograph_fr_url. https://pdf.hres.ca/dpd_pm/00048152.PDF
route.[0] Intravitreal
status_marketed_date.[0] 2018-09-27T00:00:00
drug_identification_number. 02452154
class. Human
active_ingredients_detail.[0]strength_unit_f. MG
active_ingredients_detail.[0]ingredient_f. OCRIPLASMINE
active_ingredients_detail.[0]dosage_unit. ML
active_ingredients_detail.[0]dosage_unit_f. ML
active_ingredients_detail.[0]ingredient. OCRIPLASMIN
active_ingredients_detail.[0]strength. 1.25
active_ingredients_detail.[0]strength_unit. MG
statu

In [151]:
df = pd.DataFrame(columns=['atc','status','history_date','original_market_date'])
c=0

for hit in hits:
    try:
        row = flatify(hit['_source'])
        atc = row['therapeutic_class.[0]tc_atc_number.']
        history_date = row['status_detail.[0]history_date.']
        
        if not 'status_detail.[0]original_market_date.' in hit:
            original_date = row['status_marketed_date.[0]']
        else:
            original_date = row['status_detail.[0]original_market_date.']
        
        status = row['status_detail.[0]status.']
        df.loc[c]=[atc,status,history_date,original_date]
        c+=1
    except Exception as e:
        print('{}:{} | hit# {}'.format(type(e),e.args, c))
df.head()

,atc,status,history_date,original_market_date
0,G01AF04,Marketed,2012-12-12T00:00:00,2012-12-12T00:00:00
1,C09BA03,Marketed,2002-01-25T00:00:00,2002-01-25T00:00:00
2,C07CA03,Marketed,2018-05-30T00:00:00,2018-05-30T00:00:00
3,M03BC01,Marketed,2011-04-27T00:00:00,2011-04-27T00:00:00
4,B01AB51,Marketed,2017-06-13T00:00:00,2017-06-13T00:00:00
